In [1]:
from main import RFest
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import time

In [55]:
#Checking number of cores
estimator=RFest(drop_rate=0.9999)

In [56]:
estimator.rows_range=[100000,200000,500000,700000,1000000,5000000,10000000,20000000,50000000]

In [ ]:
df=estimator.generate_data()

2018-09-01 20:13:55,474 - main - Generating dummy training durations to create a training set


In [15]:
df[1].to_csv('first_generation_1.csv')

In [22]:
algo = linear_model.LinearRegression()

In [25]:
data = pd.get_dummies(df)

In [29]:
X = (data[estimator.estimation_inputs]
             ._get_numeric_data()
             .dropna(axis=0, how='any')
             .as_matrix())
y = outputs['output'].dropna(axis=0, how='any').as_matrix()

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
algo.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [32]:

with open('coefs/lr_coefs.json', 'w') as outfile:
    json.dump([algo.intercept_]+list(algo.coef_), outfile)

In [ ]:
y_train[y_train>1]

In [52]:
y_pred_test = algo.predict(X_test)
MAPE_test = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100
y_pred_train = algo.predict(X_train[y_train>2])
#MAPE_train = np.mean(np.abs((y_train - y_pred_train) / y_train)) * 100

In [53]:
y_pred_train = algo.predict(X_train[y_train>2])

In [54]:
np.sqrt(mean_squared_error(y_train[y_train>2], y_pred_train))

1.5131380093727878

In [40]:
np.sqrt(mean_squared_error(y_test, y_pred_test))

0.28819250182748657

In [7]:
estimator.model_fit()

2018-09-01 17:35:30,751 - main - Generating dummy training durations to create a training set
2018-09-01 17:35:34,315 - main - Fitting LR to estimate training durations
2018-09-01 17:35:34,330 - main - Saving LR coefs in json file
2018-09-01 17:35:34,333 - main - Saving LR to LR_estimator.pkl
2018-09-01 17:35:34,340 - main - R squared on train set is 1.0
2018-09-01 17:35:34,342 - main - MAPE on train set is: 1.5967033308610918e-13
2018-09-01 17:35:34,347 - main - MAPE on test set is: 34.59030194142593
2018-09-01 17:35:34,349 - main - MSE on train set is 1.6490775148815707e-32
2018-09-01 17:35:34,352 - main - MSE on test set is 0.0009548975861206668


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [3]:
rf= RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features=5, max_leaf_nodes=5, min_impurity_decrease=0.0,
           min_impurity_split=5, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)


In [4]:
estimator.estimate_duration(np.random.rand(20000000,10),rf)


2018-09-01 18:26:16,008 - main - Loading LR coefs from json file
2018-09-01 18:26:16,024 - main - Training your model should take ~ 25.5884222205 seconds


   num_rows  num_features  n_estimators  max_depth  min_samples_split  \
0  20000000            10           100         10                  2   

   min_samples_leaf  min_weight_fraction_leaf  max_leaf_nodes  \
0                 1                       0.0               5   

   min_impurity_decrease  min_impurity_split  bootstrap  oob_score  n_jobs  \
0                    0.0                   5       True      False       8   

   max_features_5  max_features_auto  
0               1                  0  


25.588422220518776

In [18]:
X,y=np.random.rand(1000000,10),np.random.rand(1000000,1)
start_time = time.time()
rf.fit(X,y)
elapsed_time = time.time() - start_time

In [19]:
elapsed_time

1.861509084701538